# Building A Politician Trade Tracking AI Agent (using LlamaIndex, Financial Modeling Prep and Anthropic Claude)

## 1. Summary & Dependencies <br>
this notebook is designed to retrieve basicc financial information on a stock and identify U.S. politicians trading activity (Senate and House of Reps):

### Tools used:

1.   Anthropic API - Claude 3.5 Sonnett   
2.   LlamaIndex for orchestration
3.   Financial Modeling Prep API for stock and politician trading information
4.   News API for news summarization


In [1]:
!pip install llama-index-llms-anthropic -q
!pip install llama-index -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.8/195.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 20.4 MB/s eta 0:00:00


In [ ]:
from llama_index.llms.anthropic import Anthropic
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool

import nest_asyncio

nest_asyncio.apply()

## 2. Instantiate LLM API Keys

Step 1: Instantiate Baseline LLM to be used


*   Experiment 1: Use Claude Sonnet for tool use, code generation and parsing
*   Experiment 2: Use OpenAI o1 models to test reasoning capabilities

In [ ]:
from google.colab import userdata

CLAUDE_API_KEY = userdata.get('ANTHROPIC_API_KEY')
OAI_API_KEY = userdata.get('OPENAI_API_KEY')
FMP_API_KEY = userdata.get('FINANCIAL_MODELING_PREP_API_KEY')

### 2.1. OpenAI Agent

In [ ]:
openai_llm = OpenAI(model="gpt-4o-mini", api_key=OAI_API_KEY)

### 2.2. Anthropic/Claude Agent

In [ ]:
anthropic_llm = Anthropic(model="claude-3-5-sonnet-20240620", api_key=CLAUDE_API_KEY)

## 3. Define Tools for LLM Use

### 3.1. Retrieve Stock Profile: get_company_profile

In [ ]:
import os
import requests

def get_company_profile(symbol):
  """
  Fetch basic financial information for the given company symbol such as the␣
  industry, the sector, the name of the company, and the market capitalization.
  """
  url = f"https://financialmodelingprep.com/api/v3/profile/{symbol}?apikey={FMP_API_KEY}"
  response = requests.get(url)
  data = response.json()
  try:
    results = data[0]
    financials = {
    "symbol": results["symbol"],
    "companyName": results["companyName"],
    "marketCap": results["mktCap"],
    "industry": results["industry"],
    "sector": results["sector"],
    "website": results["website"],
    "beta":results["beta"],
    "price":results["price"],
    "lastDiv":results["lastDiv"],
    "range":results["range"],
    "address":results["address"],
    "city":results["city"],
    "state":results["state"],
    "zip":results["zip"],
    "country":results["country"],
    "description":results["description"],
    }
    return financials
  except (IndexError, KeyError):
    return {"error": f"Invalid symbol or data not available for {symbol}"}

## DATA FORMAT OF THE STOCK PROFILE FULL DATA ENDPOINT - FREE ENDPOINT (v3)
# [
# 	{
# 		"symbol": "AAPL",
# 		"price": 178.72,
# 		"beta": 1.286802,
# 		"volAvg": 58405568,
# 		"mktCap": 2794144143933,
# 		"lastDiv": 0.96,
# 		"range": "124.17-198.23",
# 		"changes": -0.13,
# 		"companyName": "Apple Inc.",
# 		"currency": "USD",
# 		"cik": "0000320193",
# 		"isin": "US0378331005",
# 		"cusip": "037833100",
# 		"exchange": "NASDAQ Global Select",
# 		"exchangeShortName": "NASDAQ",
# 		"industry": "Consumer Electronics",
# 		"website": "https://www.apple.com",
# 		"description": "Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. It also sells various related services. In addition, the company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; AirPods Max, an over-ear wireless headphone; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, HomePod, and iPod touch. Further, it provides AppleCare support services; cloud services store services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. Additionally, the company offers various services, such as Apple Arcade, a game subscription service; Apple Music, which offers users a curated listening experience with on-demand radio stations; Apple News+, a subscription news and magazine service; Apple TV+, which offers exclusive original content; Apple Card, a co-branded credit card; and Apple Pay, a cashless payment service, as well as licenses its intellectual property. The company serves consumers, and small and mid-sized businesses; and the education, enterprise, and government markets. It distributes third-party applications for its products through the App Store. The company also sells its products through its retail and online stores, and direct sales force; and third-party cellular network carriers, wholesalers, retailers, and resellers. Apple Inc. was incorporated in 1977 and is headquartered in Cupertino, California.",
# 		"ceo": "Mr. Timothy D. Cook",
# 		"sector": "Technology",
# 		"country": "US",
# 		"fullTimeEmployees": "164000",
# 		"phone": "408 996 1010",
# 		"address": "One Apple Park Way",
# 		"city": "Cupertino",
# 		"state": "CA",
# 		"zip": "95014",
# 		"dcfDiff": 4.15176,
# 		"dcf": 150.082,
# 		"image": "https://financialmodelingprep.com/image-stock/AAPL.png",
# 		"ipoDate": "1980-12-12",
# 		"defaultImage": false,
# 		"isEtf": false,
# 		"isActivelyTrading": true,
# 		"isAdr": false,
# 		"isFund": false
# 	}
# ]


### 3.2. Senate Trading Actvity: get_senate_trades

In [ ]:
import requests

def get_senate_trades(symbol, limit=10):
    """
    Track the trading activity of US Senators and identify potential conflicts of interest.
    This function provides a list of up to 10 trades that have been made by US Senators for a given stock symbol.
    """
    url = f"https://financialmodelingprep.com/api/v4/senate-trading?symbol={symbol}&apikey={FMP_API_KEY}"
    response = requests.get(url)
    data = response.json()

    if not data or not isinstance(data, list):
        return {"error": f"Invalid symbol or data not available for {symbol}"}

    trades = []
    for trade in data[:limit]:  # Limit to the first 10 results
        try:
            senate_trade = {
                "symbol": trade["symbol"],
                "assetName": trade["assetDescription"],
                "assetType": trade["assetType"],
                "firstName": trade["firstName"],
                "lastName": trade["lastName"],
                "office": trade["office"],
                "link": trade["link"],
                "date": trade["dateRecieved"],
                "transactionDate": trade["transactionDate"],
                "owner": trade["owner"],
                "type": trade["type"],
                "amount": trade["amount"],
                "comment": trade["comment"],
            }
            trades.append(senate_trade)
        except KeyError as e:
            print(f"Warning: Missing key {e} in trade data")

    return trades if trades else {"error": f"No valid trade data available for {symbol}"}


## Format of the endpoint output:
# [
# 	{
# 		"firstName": "Daniel S",
# 		"lastName": "Sullivan",
# 		"office": "Sullivan, Dan (Senator)",
# 		"link": "https://efdsearch.senate.gov/search/view/ptr/f9252211-a695-4c37-869a-0e223fce8d88/",
# 		"dateRecieved": "2023-09-21",
# 		"transactionDate": "2023-08-22",
# 		"owner": "Joint",
# 		"assetDescription": "Apple Inc",
# 		"assetType": "Stock",
# 		"type": "Sale (Full)",
# 		"amount": "$15,001 - $50,000",
# 		"comment": "Sale part of investment strategy of 3rd party investment professional to exchange individual stocks received after death of filer’s parents for EIFs.",
# 		"symbol": "AAPL"
# 	}
# ]

### 3.3. House of Reps Trading Activity: get_house_trades

In [ ]:
import requests

def get_house_trades(symbol, limit=10):
    """
    Track the trading activity of US House Representatives and identify potential conflicts of interest.
    This function provides a list of up to 10 trades that have been made by US House Representatives for a given stock symbol.
    """
    url = f"https://financialmodelingprep.com/api/v4/senate-disclosure?symbol={symbol}&apikey={FMP_API_KEY}"
    response = requests.get(url)
    data = response.json()

    if not data or not isinstance(data, list):
        return {"error": f"Invalid symbol or data not available for {symbol}"}

    trades = []
    for trade in data[:limit]:  # Limit to the first 10 results
        try:
            house_trade = {
                "ticker": trade["ticker"],
                "assetName": trade["assetDescription"],
                "representative": trade["representative"],
                "district": trade["district"],
                "link": trade["link"],
                "date": trade["disclosureDate"],
                "transactionDate": trade["transactionDate"],
                "owner": trade["owner"],
                "type": trade["type"],
                "amount": trade["amount"],
                "capitalGainsOver200USD": trade["capitalGainsOver200USD"],
            }
            trades.append(house_trade)
        except KeyError as e:
            print(f"Warning: Missing key {e} in trade data")

    return trades if trades else {"error": f"No valid trade data available for {symbol}"}


## Format of the endpoint output:
# [
# 	{
# 		"disclosureYear": "2023",
# 		"disclosureDate": "2023-08-23",
# 		"transactionDate": "2023-08-07",
# 		"owner": "self",
# 		"ticker": "AAPL",
# 		"assetDescription": "Apple Inc.",
# 		"type": "purchase",
# 		"amount": "$1,001 - $15,000",
# 		"representative": "Michael Patrick Guest",
# 		"district": "MS03",
# 		"link": "https://disclosures-clerk.house.gov/public_disc/ptr-pdfs/2023/20023442.pdf",
# 		"capitalGainsOver200USD": "False"
# 	}
# ]

### 3.4 Test Tools and Functions

In [ ]:
get_company_profile("NVDA")

{'symbol': 'NVDA',
 'companyName': 'NVIDIA Corporation',
 'marketCap': 3196013700000,
 'industry': 'Semiconductors',
 'sector': 'Technology',
 'website': 'https://www.nvidia.com',
 'beta': 1.669,
 'price': 130.29,
 'lastDiv': 0.04,
 'range': '39.23-140.76',
 'address': '2788 San Tomas Expressway',
 'city': 'Santa Clara',
 'state': 'CA',
 'zip': '95051',
 'country': 'US',
 'description': "NVIDIA Corporation provides graphics, and compute and networking solutions in the United States, Taiwan, China, and internationally. The company's Graphics segment offers GeForce GPUs for gaming and PCs, the GeForce NOW game streaming service and related infrastructure, and solutions for gaming platforms; Quadro/NVIDIA RTX GPUs for enterprise workstation graphics; vGPU software for cloud-based visual and virtual computing; automotive platforms for infotainment systems; and Omniverse software for building 3D designs and virtual worlds. Its Compute & Networking segment provides Data Center platforms and 

In [ ]:
get_senate_trades("META")

[{'symbol': 'META',
  'assetName': 'Meta Platforms Inc',
  'assetType': 'Stock',
  'firstName': 'Tommy',
  'lastName': 'Tuberville',
  'office': 'Tommy Tuberville',
  'link': 'https://efdsearch.senate.gov/search/view/ptr/3dabcae6-b7fd-4dd0-9ece-e936d07def11/',
  'date': '2024-02-14',
  'transactionDate': '2024-01-22',
  'owner': 'Joint',
  'type': 'Sale',
  'amount': '$1,001 - $15,000',
  'comment': ''},
 {'symbol': 'META',
  'assetName': 'Meta Platforms Inc',
  'assetType': 'Stock',
  'firstName': 'Tommy',
  'lastName': 'Tuberville',
  'office': 'Tommy Tuberville',
  'link': 'https://efdsearch.senate.gov/search/view/ptr/3dabcae6-b7fd-4dd0-9ece-e936d07def11/',
  'date': '2024-02-14',
  'transactionDate': '2024-01-22',
  'owner': 'Joint',
  'type': 'Sale',
  'amount': '$1,001 - $15,000',
  'comment': ''},
 {'symbol': 'META',
  'assetName': 'Meta Platforms, Inc. - Class A Common Stock',
  'assetType': 'Stock',
  'firstName': 'Tommy',
  'lastName': 'Tuberville',
  'office': 'Tommy Tubervi

In [ ]:
get_house_trades("META")

[{'ticker': 'META',
  'assetName': 'Meta Platforms, Inc. - Class A Common Stock',
  'representative': 'Josh Gottheimer',
  'district': 'NJ05',
  'link': 'https://disclosures-clerk.house.gov/public_disc/ptr-pdfs/2024/20025953.pdf',
  'date': '2024-10-03',
  'transactionDate': '2024-09-25',
  'owner': 'Joint',
  'type': 'Sale (Partial)',
  'amount': '$1,001 - $15,000',
  'capitalGainsOver200USD': 'False'},
 {'ticker': 'META',
  'assetName': 'Meta Platforms, Inc. - Class A Common Stock',
  'representative': 'John James',
  'district': 'MI10',
  'link': 'https://disclosures-clerk.house.gov/public_disc/ptr-pdfs/2024/20025799.pdf',
  'date': '2024-09-06',
  'transactionDate': '2024-09-04',
  'owner': '',
  'type': 'Sale (Full)',
  'amount': '$1,001 - $15,000',
  'capitalGainsOver200USD': 'False'},
 {'ticker': 'META',
  'assetName': 'Meta Platforms, Inc. - Class A Common Stock',
  'representative': 'John James',
  'district': 'MI10',
  'link': 'https://disclosures-clerk.house.gov/public_disc/

## 4. Create AI Agent

### 4.1. Convert Functions to Tools for LLM Use

In [ ]:
company_profile_tool = FunctionTool.from_defaults(fn=get_company_profile)
senate_trades_tool = FunctionTool.from_defaults(fn=get_senate_trades)
house_trades_tool = FunctionTool.from_defaults(fn=get_house_trades)

### 4.2 Create an Anthropic Claude Agent

In [ ]:
from llama_index.core.agent import FunctionCallingAgent

politician_trading_agent = FunctionCallingAgent.from_tools(
    [company_profile_tool, senate_trades_tool, house_trades_tool],
    llm=anthropic_llm,
    verbose=True,
    allow_parallel_tool_calls=True,
)

## 5. Chat with Agent (Politician Trade Tracker)

In [ ]:
query= "Who is actively investing in MPC?"
response = politician_trading_agent.chat(query)
print(str(response))

> Running step d764ab22-8964-47ab-95e4-0450338e8bfe. Step input: Who is actively investing in MPC?
Added user message to memory: Who is actively investing in MPC?
=== LLM Response ===
To answer your question about who is actively investing in MPC (Marathon Petroleum Corporation), we can look at the trading activities of US Senators and House Representatives. This information can give us insight into potential insider trading or conflicts of interest. Let's use the available tools to gather this information.

First, let's check the trading activity for both the Senate and the House of Representatives for MPC stock.
=== Calling Function ===
Calling function: get_senate_trades with args: {"symbol": "MPC"}
=== Function Output ===
[{'symbol': 'MPC', 'assetName': 'Marathon Petroleum Corporation', 'assetType': 'Stock', 'firstName': 'Thomas', 'lastName': 'R. Carper', 'office': 'Thomas R. Carper', 'link': 'https://efdsearch.senate.gov/search/view/ptr/bc6505f6-4219-4ce3-b146-a9faaabda324/', 'dat

In [ ]:
query= "Who is actively investing in both NVDA and META?"
response = politician_trading_agent.chat(query)
print(str(response))

> Running step dbf7f57d-3d11-4f66-a753-1a1c71d5e349. Step input: Who is actively investing in both NVDA and META?
Added user message to memory: Who is actively investing in both NVDA and META?
=== LLM Response ===
To answer your question about who is actively investing in both NVDA (NVIDIA Corporation) and META (Meta Platforms, Inc.), we'll need to check the trading activities of US Senators and House Representatives for both stocks. Let's start with NVDA, which we've already looked at, and then check META.

For NVDA:
=== Calling Function ===
Calling function: get_senate_trades with args: {"symbol": "NVDA"}
=== Function Output ===
[{'symbol': 'NVDA', 'assetName': 'NVIDIA Corporation - Common Stock', 'assetType': 'Stock', 'firstName': 'Tommy', 'lastName': 'Tuberville', 'office': 'Tommy Tuberville', 'link': 'https://efdsearch.senate.gov/search/view/ptr/dda61a64-82ed-4cc8-87e7-ef1d256c6565/', 'date': '2024-05-14', 'transactionDate': '2024-04-30', 'owner': 'Joint', 'type': 'Sale (Full)', '

In [ ]:
query= "What stocks has Josh Gottheimer traded?"
response = politician_trading_agent.chat(query)
print(str(response))

> Running step 443227d6-834e-4eb3-94cb-585f9084033f. Step input: What stocks has Josh Gottheimer traded?
Added user message to memory: What stocks has Josh Gottheimer traded?
=== LLM Response ===
To answer your question about which stocks Josh Gottheimer has traded, we'll need to look at his trading activity across various stocks. Since we don't have a comprehensive list of all his trades, we'll need to check for each stock individually. Let's start with the stocks we've already seen (NVDA and META) and then we can check a few other common tech stocks to get a broader picture.

We've already seen some of Josh Gottheimer's trades in META, but let's double-check for both NVDA and META:
=== Calling Function ===
Calling function: get_house_trades with args: {"symbol": "NVDA"}
=== Function Output ===
[{'ticker': 'NVDA', 'assetName': 'NVIDIA Corporation - Common Stock', 'representative': 'Pete Sessions', 'district': 'TX17', 'link': 'https://disclosures-clerk.house.gov/public_disc/ptr-pdfs/20

In [ ]:
query= "What stocks has Nancy Pelosi invested in?"
response = politician_trading_agent.chat(query)
print(str(response))

> Running step 5f9ca81f-6032-42e0-b7d0-d16a7d5f06ab. Step input: What stocks has Nancy Pelosi invested in?
Added user message to memory: What stocks has Nancy Pelosi invested in?
=== LLM Response ===
To answer your question about Nancy Pelosi's stock investments, we'll need to check her trading activity across various stocks. Let's use the available tools to gather this information for some of the most commonly traded stocks.
=== Calling Function ===
Calling function: get_house_trades with args: {"symbol": "AAPL"}
=== Function Output ===
[{'ticker': 'AAPL', 'assetName': 'Apple Inc. - Common Stock', 'representative': 'Josh Gottheimer', 'district': 'NJ05', 'link': 'https://disclosures-clerk.house.gov/public_disc/ptr-pdfs/2024/20025953.pdf', 'date': '2024-10-03', 'transactionDate': '2024-09-12', 'owner': 'Joint', 'type': 'Purchase', 'amount': '$1,001 - $15,000', 'capitalGainsOver200USD': 'False'}, {'ticker': 'AAPL', 'assetName': 'Apple Inc. - Common Stock', 'representative': 'Morgan McGar